# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 11:15AM,244255,22,624415,"NBTY Global, Inc.",Released
1,Aug 16 2022 11:13AM,244254,102,WT5290 Out,"Senseonics, Incorporated",Released
2,Aug 16 2022 11:02AM,244253,10,MSP213131,"Methapharm, Inc.",Released
3,Aug 16 2022 11:02AM,244253,10,MSP213139,"Methapharm, Inc.",Released
4,Aug 16 2022 11:02AM,244250,10,MSP213132,"Methapharm, Inc.",Released
5,Aug 16 2022 11:02AM,244250,10,MSP213149,"Methapharm, Inc.",Released
6,Aug 16 2022 11:02AM,244250,10,MSP213152,"Methapharm, Inc.",Released
7,Aug 16 2022 11:01AM,244251,22,624412,"NBTY Global, Inc.",Released
8,Aug 16 2022 10:59AM,244249,10,MSP213126,"Methapharm, Inc.",Released
9,Aug 16 2022 10:59AM,244249,10,MSP213127,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,244250,Released,3
33,244251,Released,1
34,244253,Released,2
35,244254,Released,1
36,244255,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244250,NaN,3.0
244251,NaN,1.0
244253,NaN,2.0
244254,NaN,1.0
244255,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244147,33.0,12.0
244164,0.0,1.0
244183,0.0,1.0
244189,0.0,1.0
244191,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244147,33,12
244164,0,1
244183,0,1
244189,0,1
244191,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244147,33,12
1,244164,0,1
2,244183,0,1
3,244189,0,1
4,244191,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244147,33,12
1,244164,,1
2,244183,,1
3,244189,,1
4,244191,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 11:15AM,244255,22,"NBTY Global, Inc."
1,Aug 16 2022 11:13AM,244254,102,"Senseonics, Incorporated"
2,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc."
4,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc."
7,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc."
8,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc."
29,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC"
41,Aug 16 2022 9:58AM,244243,20,"Exact-Rx, Inc."
46,Aug 16 2022 9:58AM,244247,10,Eywa Pharma Inc.
62,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 16 2022 11:15AM,244255,22,"NBTY Global, Inc.",,1
1,Aug 16 2022 11:13AM,244254,102,"Senseonics, Incorporated",,1
2,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",,2
3,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",,3
4,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",,1
5,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc.",,21
6,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC",6,6
7,Aug 16 2022 9:58AM,244243,20,"Exact-Rx, Inc.",,5
8,Aug 16 2022 9:58AM,244247,10,Eywa Pharma Inc.,,16
9,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:15AM,244255,22,"NBTY Global, Inc.",1,
1,Aug 16 2022 11:13AM,244254,102,"Senseonics, Incorporated",1,
2,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2,
3,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3,
4,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1,
5,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc.",21,
6,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC",6,6
7,Aug 16 2022 9:58AM,244243,20,"Exact-Rx, Inc.",5,
8,Aug 16 2022 9:58AM,244247,10,Eywa Pharma Inc.,16,
9,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:15AM,244255,22,"NBTY Global, Inc.",1,
1,Aug 16 2022 11:13AM,244254,102,"Senseonics, Incorporated",1,
2,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2,
3,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3,
4,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:15AM,244255,22,"NBTY Global, Inc.",1.0,NaN
1,Aug 16 2022 11:13AM,244254,102,"Senseonics, Incorporated",1.0,NaN
2,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2.0,NaN
3,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3.0,NaN
4,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:15AM,244255,22,"NBTY Global, Inc.",1.0,0.0
1,Aug 16 2022 11:13AM,244254,102,"Senseonics, Incorporated",1.0,0.0
2,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2.0,0.0
3,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3.0,0.0
4,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3175065,144.0,2.0
102,244254,1.0,0.0
15,732653,73.0,51.0
16,976797,4.0,0.0
19,488370,1.0,1.0
20,976799,19.0,33.0
21,732583,3.0,0.0
22,488506,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3175065,144.0,2.0
1,102,244254,1.0,0.0
2,15,732653,73.0,51.0
3,16,976797,4.0,0.0
4,19,488370,1.0,1.0
5,20,976799,19.0,33.0
6,21,732583,3.0,0.0
7,22,488506,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,144.0,2.0
1,102,1.0,0.0
2,15,73.0,51.0
3,16,4.0,0.0
4,19,1.0,1.0
5,20,19.0,33.0
6,21,3.0,0.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,144.0
1,102,Released,1.0
2,15,Released,73.0
3,16,Released,4.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20,21,22
Status,,,,,,,,
Executing,2.0,0.0,51.0,0.0,1.0,33.0,0.0,0.0
Released,144.0,1.0,73.0,4.0,1.0,19.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20,21,22
0,Executing,2.0,0.0,51.0,0.0,1.0,33.0,0.0,0.0
1,Released,144.0,1.0,73.0,4.0,1.0,19.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20,21,22
0,Executing,2.0,0.0,51.0,0.0,1.0,33.0,0.0,0.0
1,Released,144.0,1.0,73.0,4.0,1.0,19.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()